In [19]:
from pymongo import MongoClient
import datetime
import pprint
import re
from pathlib import Path
import pprint
import json

In [20]:
from pdf_parser_ver2 import PdfParser, DirectoryPdfParser

## データベースの設定 

In [21]:
client = MongoClient('localhost', 27017)
db = client["db"]
collection = db["papers"]

## データベースへの登録 

ディレクトリから読み込み，データベースへ登録

In [22]:
start_patterns = {"序論":re.compile("[1-9]*( |　)*(背景|はじめに|Abstract|序論|概要|Introduction|序)"), 
                  "結論":re.compile("[2-9]+( |　)*(おわりに|結論|まとめ|むすび|Conclusion)"),
                  "全部":re.compile(".*") # 全てのものにマッチング
                 }  # これが当てはまらないものも多い
end_patterns = {"序論":re.compile("[2-9]+( |　)*(関連研究|提案手法|従来手法|従来研究)?"), #とりあえず2章以降はダメ
                "結論":re.compile("参考文献|謝辞"),
                "全部":None
               }  # これが当てはまらないものも多い
max_iter = 50
parse_page_numbers = None

### jsonファイルからディレクトリ情報を取得 

In [25]:
#json_path = Path('./paper_directory.json')
#json_path = Path('./paper_directory_ver2.json')
json_path = Path('./paper_directory_ver3.json')

with open(json_path, encoding="utf-8") as f:
    paper_directory_information = json.load(f)
    
dir_path_list = [Path(str_path) for str_path in paper_directory_information["directories"]]
conference_name_list = paper_directory_information["conference_names"]
title_position_number_list = paper_directory_information["title_position_numbers"]
date_list = paper_directory_information["dates"]
print(dir_path_list, conference_name_list, title_position_number_list, date_list)

[PosixPath('/mnt/umenas_root/SSII/SSII 2015/papers/IS1'), PosixPath('/mnt/umenas_root/SSII/SSII 2015/papers/IS2'), PosixPath('/mnt/umenas_root/SSII/SSII 2015/papers/IS3'), PosixPath('/mnt/umenas_root/SSII/SSII 2016/pdf'), PosixPath('/mnt/umenas_root/SSII/SSII 2017/pdf'), PosixPath('/mnt/umenas_root/SSII/SSII 2018/pdf'), PosixPath('/mnt/umenas_root/SSII/SSII 2019/pdf'), PosixPath('/mnt/umenas_root/ViEW/ViEW2015/docs'), PosixPath('/mnt/umenas_root/MIRU/MIRU 2015/ea'), PosixPath('/mnt/umenas_root/MIRU/MIRU 2016/pdf'), PosixPath('/mnt/umenas_root/MIRU/MIRU 2019/ea'), PosixPath('/mnt/umenas_root/ロボティクスシンポジア/2015/paper'), PosixPath('/mnt/umenas_root/ロボティクスシンポジア/2016/paper'), PosixPath('/mnt/umenas_root/ロボティクスシンポジア/2017/paper'), PosixPath('/mnt/umenas_root/ロボティクスシンポジア/2018'), PosixPath('/mnt/umenas_root/ロボティクスシンポジア/2019/rs2019data/paper'), PosixPath('/mnt/umenas_root/ロボティクスシンポジア/2020/paper'), PosixPath('/mnt/umenas_root/ROBOMECH（ROBOMEC）/2017/PDF'), PosixPath('/mnt/umenas_root/ROBOMECH（ROBOME

In [26]:
for dir_path, conference_name, title_position_number, date_str in zip(dir_path_list, 
                                                                      conference_name_list, 
                                                                      title_position_number_list, 
                                                                      date_list):
    
    date = datetime.datetime(date_str[0], date_str[1], 1, 00, 00, 00),
    
    pdf_parser = PdfParser(conference_name=conference_name,
                           start_patterns=start_patterns,
                           end_patterns=end_patterns,
                           title_position_number=title_position_number,
                           parse_page_numbers=parse_page_numbers,
                           date=date
                           )



    dir_pdf_parser = DirectoryPdfParser(dir_path=dir_path,
                                        pdf_parser=pdf_parser,
                                        max_iter = max_iter
                                        )
#     try:
    for dict_list in dir_pdf_parser.parse_dict_list():
        #result = collection.insert_many(dict_list)  # これは用いない

        for one_dict in dict_list:
            find_result = collection.find_one(filter={"$and":[{"conf_name":one_dict["conf_name"]},
                                                              {"pdf_name":one_dict["pdf_name"]}]})  # 同じ学会名とファイル名があるかどうかチェック
            if find_result is None:  # 検索結果が返ってこなかった場合
                 result = collection.insert_one(one_dict)
#     except:
#         pass

100%|██████████| 490/490 [1:41:26<00:00, 12.42s/it]      


In [27]:
counter = 0
for dict in collection.find():
    counter += 1

print(counter)

2942


In [18]:
find_one = collection.find_one()
print(find_one)

{'_id': ObjectId('5ee50e7b521d6a64070a5ae3'), 'pdf_name': '0002', 'paper_title': '(cid:18644)(cid:18645)(cid:18636)(cid:18612)(cid:18610)(cid:18611)(cid:18610)(cid:18627)(cid:18629)(cid:18613)(cid:18630)(cid:18612)(cid:18607)(cid:18613)', 'content': {'序論': '', '結論': ''}, 'date': '(datetime.datetime(2010, 9, 1, 0, 0),)', 'conf_name': 'robotgakkai2019'}


In [8]:
find_one = collection.find_one(filter={"pdf_name":"0005"})
print(find_one)

{'_id': ObjectId('5ee50e7b521d6a64070a5ae5'), 'pdf_name': '0005', 'paper_title': '(cid:18644)(cid:18645)(cid:18636)(cid:18612)(cid:18610)(cid:18611)(cid:18610)(cid:18627)(cid:18629)(cid:18611)(cid:18643)(cid:18612)(cid:18607)(cid:18612)', 'content': {'序論': '', '結論': ''}, 'date': '(datetime.datetime(2010, 9, 1, 0, 0),)', 'conf_name': 'robotgakkai2019'}


## コレクションの削除 

軽率に削除してはいけない

In [17]:
collection.drop()

## データベースの削除 

In [18]:
client.drop_database("my_db")